# Servier Drugs exploratory data analysis
#### Salmane KECHKAR
###### GCP Cloud Architect - Data / ML Engineer

## Setup

In [1]:
! gcloud config set account kechkar.s@sfeir.com
! gcloud auth application-default login


If you have a compatible Python interpreter installed, you can use it by setting
the CLOUDSDK_PYTHON environment variable to point to it.

Updated property [core/account].

If you have a compatible Python interpreter installed, you can use it by setting
the CLOUDSDK_PYTHON environment variable to point to it.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=lIZj7Olw3CiuVHKFGuIvrsQEjie9X1&access_type=offline&code_challenge=yIo3SDznb5hxXXVXKLJmTiDJqc-HejUeLMIfYtp17Xc&code_challenge_method=S256


Credentials saved to file: [/Users/salmane/.config/gcloud/applic

In [2]:
# Retrieve and set PROJECT_ID and REGION environment variables.
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[-1]
REGION = !(gcloud config get-value compute/region)
REGION = REGION[-1]

GCS_BUCKET = f"{PROJECT_ID}-sk-bucket"

In [3]:
# Create a globally unique Google Cloud Storage bucket for artifact storage.
!gsutil mb -l $REGION gs://$GCS_BUCKET


If you have a compatible Python interpreter installed, you can use it by setting
the CLOUDSDK_PYTHON environment variable to point to it.

Creating gs://sfeir-innovation-sk-bucket/...
ServiceException: 409 A Cloud Storage bucket named 'sfeir-innovation-sk-bucket' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [7]:
# Upload input data to bucket
!gsutil cp -r ../data/raw_data/* gs://$GCS_BUCKET/input


If you have a compatible Python interpreter installed, you can use it by setting
the CLOUDSDK_PYTHON environment variable to point to it.

Copying file://../data/raw_data/clinical_trials.csv [Content-Type=text/csv]...
Copying file://../data/raw_data/drugs.csv [Content-Type=text/csv]...            
Copying file://../data/raw_data/pubmed.csv [Content-Type=text/csv]...           
Copying file://../data/raw_data/pubmed.json [Content-Type=application/json]...  
/ [4 files][  3.9 KiB/  3.9 KiB]                                                
Operation completed over 4 objects/3.9 KiB.                                      


In [4]:
import yaml
import gcsfs
import polars as pl

# Load credentials
credentials = "/Users/salmane/.config/gcloud/application_default_credentials.json"

# Initialize the GCS file system with the credentials
fs = gcsfs.GCSFileSystem(project=PROJECT_ID, token=credentials)

## Load files

In [5]:
# paths

DRUGS_FILE_PATH = f"gs://{GCS_BUCKET}/input/drugs.csv"
PUBMED_CSV_FIL_PATH = f"gs://{GCS_BUCKET}/input/pubmed.csv"
PUBMED_JSON_FIL_PATH = f"gs://{GCS_BUCKET}/input/pubmed.json"
CLINICAL_TRIAL_FILE_PATH = f"gs://{GCS_BUCKET}/input/clinical_trials.csv"


# open drugs file
with fs.open(DRUGS_FILE_PATH, 'rb') as file:
    # Read the CSV file using Polars
    drugs = pl.read_csv(file)

# open clinical trial file
with fs.open(CLINICAL_TRIAL_FILE_PATH, 'rb') as file:
    clinical_trials = pl.read_csv(file)
    
# open pubmed csv file
with fs.open(PUBMED_CSV_FIL_PATH, 'rb') as file:
    pubmed_csv = pl.read_csv(file)
    
# open pubmed csv file (issue detected with comma trailing)
with fs.open(PUBMED_JSON_FIL_PATH, 'r') as file:
    pubmed_json = pl.DataFrame(yaml.safe_load(file))


## Data Cleansing

### Clinical trials

Some date inconsistancies are identified in clinical_trials dataframe. Let's stick on 'YYYY-MM-DD' date format

In [6]:
# Define date formats to try
date_formats = ["%F", "%-d %B %Y", "%d/%m/%Y"]

# Parse dates in different formats and coalesce them into a single column
clinical_trials = clinical_trials.with_columns(
    pl.coalesce([
        clinical_trials["date"].str.to_date(fmt, strict=False) for fmt in date_formats
    ]).alias("date")
)

Also special characters are populating both columns journal and scientific title to be removed

In [8]:
# Define the regular expression pattern
REGEX_PATTERN = r"(\\x[a-f0-9]+)+"

# Define a function to remove escape sequences
def remove_escape_sequences(series, pattern):
    return series.str.replace(pattern, "")

# Apply the function to the relevant columns
clinical_trials = clinical_trials.with_columns([
    remove_escape_sequences(clinical_trials["journal"], REGEX_PATTERN).alias("journal"),
    remove_escape_sequences(clinical_trials["scientific_title"], REGEX_PATTERN).alias("scientific_title")
])

In [9]:
clinical_trials

id,scientific_title,date,journal
str,str,date,str
"""NCT01967433""","""Use of Diphenhydramine as an A…",2020-01-01,"""Journal of emergency nursing"""
"""NCT04189588""","""Phase 2 Study IV QUZYTTIR™ (Ce…",2020-01-01,"""Journal of emergency nursing"""
"""NCT04237090""",""" """,2020-01-01,"""Journal of emergency nursing"""
"""NCT04237091""","""Feasibility of a Randomized Co…",2020-01-01,"""Journal of emergency nursing"""
"""NCT04153396""","""Preemptive Infiltration With B…",2020-01-01,"""Hôpitaux Universitaires de Gen…"
"""NCT03490942""","""Glucagon Infusion in T1D Patie…",2020-05-25,""""""
"""""","""Glucagon Infusion in T1D Patie…",2020-05-25,"""Journal of emergency nursing"""
"""NCT04188184""","""Tranexamic Acid Versus Epineph…",2020-04-27,"""Journal of emergency nursing"""


Later on, In order to establish the link with pubmed dataframe let create a new column inside ``clinical_trials`` and call it `type` and set a column order

In [10]:
# Create a new column type
clinical_trials = clinical_trials.with_columns(
    [pl.lit("clinical_trial").alias("type")]
)

# Set column order
clinical_trials = clinical_trials.select([
     "type","id","scientific_title","journal","date"
])

In [11]:
clinical_trials


type,id,scientific_title,journal,date
str,str,str,str,date
"""clinical_trial""","""NCT01967433""","""Use of Diphenhydramine as an A…","""Journal of emergency nursing""",2020-01-01
"""clinical_trial""","""NCT04189588""","""Phase 2 Study IV QUZYTTIR™ (Ce…","""Journal of emergency nursing""",2020-01-01
"""clinical_trial""","""NCT04237090""",""" ""","""Journal of emergency nursing""",2020-01-01
"""clinical_trial""","""NCT04237091""","""Feasibility of a Randomized Co…","""Journal of emergency nursing""",2020-01-01
"""clinical_trial""","""NCT04153396""","""Preemptive Infiltration With B…","""Hôpitaux Universitaires de Gen…",2020-01-01
"""clinical_trial""","""NCT03490942""","""Glucagon Infusion in T1D Patie…","""""",2020-05-25
"""clinical_trial""","""""","""Glucagon Infusion in T1D Patie…","""Journal of emergency nursing""",2020-05-25
"""clinical_trial""","""NCT04188184""","""Tranexamic Acid Versus Epineph…","""Journal of emergency nursing""",2020-04-27


### PubMed

Let's impact the same cleaning above against pubmed datafram

Union csv and json pubmed's files

In [12]:
pubmed = pl.concat([pubmed_csv,pubmed_json])

In [13]:
pubmed

id,title,date,journal
str,str,str,str
"""1""","""A 44-year-old man with erythem…","""01/01/2019""","""Journal of emergency nursing"""
"""2""","""An evaluation of benadryl, pyr…","""01/01/2019""","""Journal of emergency nursing"""
"""3""","""Diphenhydramine hydrochloride …","""02/01/2019""","""The Journal of pediatrics"""
"""4""","""Tetracycline Resistance Patter…","""01/01/2020""","""Journal of food protection"""
"""5""","""Appositional Tetracycline bone…","""02/01/2020""","""American journal of veterinary…"
…,…,…,…
"""9""","""Gold nanoparticles synthesized…","""01/01/2020""","""Journal of photochemistry and …"
"""10""","""Clinical implications of umbil…","""01/01/2020""","""The journal of maternal-fetal …"
"""11""","""Effects of Topical Application…","""01/01/2020""","""Journal of back and musculoske…"


In [14]:
# format date
pubmed = pubmed.with_columns(
    pl.coalesce([
        pubmed["date"].str.to_date(fmt, strict=False) for fmt in date_formats
    ]).alias("date")
)

# rename title column
pubmed = pubmed.rename({
    "title":"scientific_title"
})

# added type column
pubmed = pubmed.with_columns(
    pl.lit("pubmed").alias("type")
)

# Reorder columns
pubmed = pubmed.select([
     "type","id","scientific_title","journal","date"
])


In [15]:
pubmed


type,id,scientific_title,journal,date
str,str,str,str,date
"""pubmed""","""1""","""A 44-year-old man with erythem…","""Journal of emergency nursing""",2019-01-01
"""pubmed""","""2""","""An evaluation of benadryl, pyr…","""Journal of emergency nursing""",2019-01-01
"""pubmed""","""3""","""Diphenhydramine hydrochloride …","""The Journal of pediatrics""",2019-01-02
"""pubmed""","""4""","""Tetracycline Resistance Patter…","""Journal of food protection""",2020-01-01
"""pubmed""","""5""","""Appositional Tetracycline bone…","""American journal of veterinary…",2020-01-02
…,…,…,…,…
"""pubmed""","""9""","""Gold nanoparticles synthesized…","""Journal of photochemistry and …",2020-01-01
"""pubmed""","""10""","""Clinical implications of umbil…","""The journal of maternal-fetal …",2020-01-01
"""pubmed""","""11""","""Effects of Topical Application…","""Journal of back and musculoske…",2020-01-01


## Tasks

**I ) Find out drugs that are mentionned in the title of both PubMed and clinical trials**

Approch:

- Cross the pubmed-clinical-trials-union with drugs
- Lower cases columns drug and title 
- shortlist drugs mentionned in titles
- output results

In [16]:
# Union pubmed with clinical trials
pubmed_clinical_trials = pl.concat([pubmed, clinical_trials])

#
output = pubmed_clinical_trials.join(
    drugs, how='cross'
).with_columns([
    pl.col("scientific_title").str.to_lowercase().str.contains(pl.col("drug").str.to_lowercase()).alias("contains") # check exisitings
]).filter(
    pl.col("contains") # keep only matchs
).select(
    pubmed_clinical_trials.columns + drugs.columns # output results
)


In [17]:
output

type,id,scientific_title,journal,date,atccode,drug
str,str,str,str,date,str,str
"""pubmed""","""1""","""A 44-year-old man with erythem…","""Journal of emergency nursing""",2019-01-01,"""A04AD""","""DIPHENHYDRAMINE"""
"""pubmed""","""2""","""An evaluation of benadryl, pyr…","""Journal of emergency nursing""",2019-01-01,"""A04AD""","""DIPHENHYDRAMINE"""
"""pubmed""","""3""","""Diphenhydramine hydrochloride …","""The Journal of pediatrics""",2019-01-02,"""A04AD""","""DIPHENHYDRAMINE"""
"""pubmed""","""4""","""Tetracycline Resistance Patter…","""Journal of food protection""",2020-01-01,"""S03AA""","""TETRACYCLINE"""
"""pubmed""","""5""","""Appositional Tetracycline bone…","""American journal of veterinary…",2020-01-02,"""S03AA""","""TETRACYCLINE"""
…,…,…,…,…,…,…
"""clinical_trial""","""NCT01967433""","""Use of Diphenhydramine as an A…","""Journal of emergency nursing""",2020-01-01,"""A04AD""","""DIPHENHYDRAMINE"""
"""clinical_trial""","""NCT04189588""","""Phase 2 Study IV QUZYTTIR™ (Ce…","""Journal of emergency nursing""",2020-01-01,"""A04AD""","""DIPHENHYDRAMINE"""
"""clinical_trial""","""NCT04237091""","""Feasibility of a Randomized Co…","""Journal of emergency nursing""",2020-01-01,"""A04AD""","""DIPHENHYDRAMINE"""


**II ) Bonus **

- Extract journal mentionning the highest number of drugs
- For given drug, find out all drugs mentionned by the same journals referenced by PubMed but not clinical trials

In [18]:
top_journals = output.group_by("journal").agg(
    pl.col("drug").unique().count().alias("distinct_drug_count")
).filter(
    pl.col("distinct_drug_count") == pl.col("distinct_drug_count").max()
)

# write json file
top_journals.write_json('../data/output/output1.json')

In [19]:
drug_name = "ATROPINE"

drug_journals = output.filter(
    pl.col("type") == "pubmed"
).filter(
    pl.col("drug") == drug_name
)["journal"].to_list()

codrugs = output.filter(
    pl.col("journal").is_in(drug_journals)
).select(
    pl.col("journal").alias("journal_list"),
    pl.col("drug")
).group_by(
    pl.col("drug")
).all()
codrugs.write_json('../data/output/output2.json')